In [1]:
import pandas as pd
import pickle
import numpy as np
from pandas_summary import DataFrameSummary
import csv

In [2]:
#data = pd.read_csv('../../../data/hcup/nis/all_year_combination/cleaned_for_repcv.csv', quoting = csv.QUOTE_ALL, dtype = str)
data = pd.read_pickle('../../../data/hcup/nis/all_year_combination/cleaned_for_repcv.pickle')

## Create season variable

In [5]:
cm_vars = ['CCI']

demo_vars = ['AGE', 'FEMALE', 'RACE_1', 'RACE_2', 'RACE_3', 'RACE_4', 'RACE_5', 'RACE_6', 'PAY1_1', 
                 'PAY1_2', 'PAY1_3', 'PAY1_4', 'PAY1_5', 'PAY1_6', 'ZIPINC_QRTL_1', 'ZIPINC_QRTL_2', 
                 'ZIPINC_QRTL_3', 'ZIPINC_QRTL_4', 'PL_NCHS']

admin_vars = ['ASEASON_SPRING', 'ASEASON_SUMMER', 'ASEASON_FALL', 'ASEASON_WINTER', 'YEAR_2007', 'YEAR_2008',
                 'YEAR_2009', 'YEAR_2010', 'YEAR_2011', 'YEAR_2012', 'YEAR_2013', 'YEAR_2014', 'YEAR_2015', 'AWEEKEND', 'ELECTIVE']

hospital_vars = ['TOTAL_DISC', 'HOSP_BEDSIZE', 'HOSP_REGION_1', 'HOSP_REGION_2', 'HOSP_REGION_3', 'HOSP_REGION_4', 'H_CONTRL_1', 'H_CONTRL_2', 
                 'H_CONTRL_3', 'HOSP_LOCTEACH_1', 'HOSP_LOCTEACH_2', 'HOSP_LOCTEACH_3', 'HCUP_ED_0', 'HCUP_ED_1', 
                 'HCUP_ED_2', 'HCUP_ED_3', 'HCUP_ED_4']

ndx = ['NDX']

npr = ['NPR']

nchronic = ['NCHRONIC']

# variables of the best diagnose system
with open('../tests/feature_selection/dxccs.pkl', 'rb') as f:
    dx_vars = pickle.load(f)

# variables of the best procedure coding system
with open('../tests/feature_selection/prccs.pkl', 'rb') as f:
    pr_vars = pickle.load(f)

names = cm_vars + demo_vars + admin_vars + hospital_vars + ndx + npr + nchronic + dx_vars + pr_vars
num_vars = ['HOSP_BEDSIZE', 'AGE', 'NDX', 'NPR', 'TOTAL_DISC', 'NCHRONIC',
                'PL_NCHS']

In [6]:
with open('../saved_models/input_variables.pkl', 'wb') as f:
    pickle.dump(names, f)

In [7]:
target_var = ['DIED']

In [8]:
data = data[names + target_var]

In [9]:
data.replace(['A', 'B', 'C', 'M', 'F'], np.nan, inplace = True)

In [10]:
data = data.convert_objects(convert_numeric=True)

/apps/python3/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


In [11]:
data.head()

,CCI,AGE,FEMALE,RACE_1,RACE_2,RACE_3,RACE_4,RACE_5,RACE_6,PAY1_1,PAY1_2,PAY1_3,PAY1_4,PAY1_5,PAY1_6,ZIPINC_QRTL_1,ZIPINC_QRTL_2,ZIPINC_QRTL_3,ZIPINC_QRTL_4,PL_NCHS,ASEASON_SPRING,ASEASON_SUMMER,ASEASON_FALL,ASEASON_WINTER,YEAR_2007,YEAR_2008,YEAR_2009,YEAR_2010,YEAR_2011,YEAR_2012,YEAR_2013,YEAR_2014,YEAR_2015,AWEEKEND,ELECTIVE,TOTAL_DISC,HOSP_BEDSIZE,HOSP_REGION_1,HOSP_REGION_2,HOSP_REGION_3,...,HOSP_LOCTEACH_3,HCUP_ED_0,HCUP_ED_1,HCUP_ED_2,HCUP_ED_3,HCUP_ED_4,NDX,NPR,NCHRONIC,DXCCS_131,DXCCS_249,DXCCS_107,DXCCS_157,DXCCS_2,DXCCS_122,DXCCS_257,DXCCS_59,DXCCS_259,DXCCS_244,PRCCS_216,PRCCS_225,PRCCS_54,PRCCS_222,PRCCS_224,PRCCS_37,PRCCS_65,PRCCS_223,PRCCS_231,PRCCS_63,PRCCS_58,PRCCS_193,PRCCS_174,PRCCS_4,PRCCS_64,PRCCS_70,PRCCS_39,PRCCS_173,PRCCS_108,PRCCS_71,DIED
0,2,49.0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,3.0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.0,19802,3,0,0,0,...,0,0,0,1,0,0,9,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,66.0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,5.0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0.0,7152,3,0,0,0,...,0,1,0,0,0,0,9,0,6,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,28.0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1.0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0.0,23015,3,0,0,0,...,0,0,0,1,0,0,8,0,3,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2,39.0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,23015,3,0,0,0,...,0,0,0,1,0,0,5,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,62.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,3.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,13428,2,0,0,0,...,1,0,0,1,0,0,9,1,6,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
data[data<0] = np.nan

In [13]:
DataFrameSummary(data[num_vars]).summary()

,HOSP_BEDSIZE,AGE,NDX,NPR,TOTAL_DISC,NCHRONIC,PL_NCHS
count,125276,126013,126013,126013,126013,126013,123354
mean,2.56441,58.5265,13.3371,2.1552,16425.8,6.06633,2.64304
std,0.69702,15.8922,5.61484,2.70024,17626.9,2.63135,1.55569
min,1,18,3,0,17,1,1
25%,2,50,9,0,4411,4,1
50%,3,61,13,1,9760,6,2
75%,3,70,17,3,23026,8,4
max,3,100,50,31,121668,22,6
counts,125276,126013,126013,126013,126013,126013,123354
uniques,3,83,39,32,6856,22,6


In [14]:
data[num_vars] = data[num_vars].fillna(data[num_vars].mean())

In [15]:
DataFrameSummary(data[num_vars]).summary()

,HOSP_BEDSIZE,AGE,NDX,NPR,TOTAL_DISC,NCHRONIC,PL_NCHS
count,126013,126013,126013,126013,126013,126013,126013
mean,2.56441,58.5265,13.3371,2.1552,16425.8,6.06633,2.64304
std,0.694979,15.8922,5.61484,2.70024,17626.9,2.63135,1.53919
min,1,18,3,0,17,1,1
25%,2,50,9,0,4411,4,1
50%,3,61,13,1,9760,6,2
75%,3,70,17,3,23026,8,4
max,3,100,50,31,121668,22,6
counts,126013,126013,126013,126013,126013,126013,126013
uniques,4,83,39,32,6856,22,7


pay attention for imputation with mode, which is different with mean: https://stackoverflow.com/questions/38223579/filling-nan-data-with-mode-doesnt-work-pandas

In [16]:
data = data.fillna(data.mode().iloc[0])

In [17]:
data.head()

,CCI,AGE,FEMALE,RACE_1,RACE_2,RACE_3,RACE_4,RACE_5,RACE_6,PAY1_1,PAY1_2,PAY1_3,PAY1_4,PAY1_5,PAY1_6,ZIPINC_QRTL_1,ZIPINC_QRTL_2,ZIPINC_QRTL_3,ZIPINC_QRTL_4,PL_NCHS,ASEASON_SPRING,ASEASON_SUMMER,ASEASON_FALL,ASEASON_WINTER,YEAR_2007,YEAR_2008,YEAR_2009,YEAR_2010,YEAR_2011,YEAR_2012,YEAR_2013,YEAR_2014,YEAR_2015,AWEEKEND,ELECTIVE,TOTAL_DISC,HOSP_BEDSIZE,HOSP_REGION_1,HOSP_REGION_2,HOSP_REGION_3,...,HOSP_LOCTEACH_3,HCUP_ED_0,HCUP_ED_1,HCUP_ED_2,HCUP_ED_3,HCUP_ED_4,NDX,NPR,NCHRONIC,DXCCS_131,DXCCS_249,DXCCS_107,DXCCS_157,DXCCS_2,DXCCS_122,DXCCS_257,DXCCS_59,DXCCS_259,DXCCS_244,PRCCS_216,PRCCS_225,PRCCS_54,PRCCS_222,PRCCS_224,PRCCS_37,PRCCS_65,PRCCS_223,PRCCS_231,PRCCS_63,PRCCS_58,PRCCS_193,PRCCS_174,PRCCS_4,PRCCS_64,PRCCS_70,PRCCS_39,PRCCS_173,PRCCS_108,PRCCS_71,DIED
0,2,49.0,1.0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,3.0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.0,19802,3.0,0,0,0,...,0,0,0,1,0,0,9,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1,2,66.0,1.0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,5.0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0.0,7152,3.0,0,0,0,...,0,1,0,0,0,0,9,0,6,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2,2,28.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1.0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0.0,23015,3.0,0,0,0,...,0,0,0,1,0,0,8,0,3,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3,2,39.0,1.0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,23015,3.0,0,0,0,...,0,0,0,1,0,0,5,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4,2,62.0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,3.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,13428,2.0,0,0,0,...,1,0,0,1,0,0,9,1,6,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [18]:
data.to_pickle('../../../data/hcup/nis/all_year_combination/cleaned_for_repcv_imputed.pickle')